In [162]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
from __future__ import print_function
import csv as csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#mpl.rcParams.update({'axes.labelsize': 20})
import copy
import re

In [164]:
route_df = pd.read_csv('Race_data_team_splits_checkpoints.csv', header=0)

In [165]:
route_df['team ID'] = route_df['Name'].map(lambda x: int(x[:3]))
route_df['Team Name'] = route_df['Name'].map(lambda x: x[4:].strip())
route_df['Checkpoint'] = route_df['Checkpoint'].str.replace(r"\(","")
route_df['Checkpoint'] = route_df['Checkpoint'].str.replace(r"\)","")
route_df.columns = ['Name', 'Checkpoint', 'Total Time', 'Split', 'team ID', 'Team Name']
route_df = route_df.drop('Name', 1)

In [166]:
cols = route_df.columns.tolist()
cols = cols[-2:] + cols[:-2]
route_df = route_df[cols]

In [167]:
route_df.to_csv('clean_data/team_routes.csv')

In [168]:
route_df.head(20)

,team ID,Team Name,Checkpoint,Total Time,Split
0,285,NimKik Skulls,105,0:13:04,0:13:04
1,285,NimKik Skulls,126,0:27:33,0:14:29
2,285,NimKik Skulls,143,0:45:17,0:17:44
3,285,NimKik Skulls,103,1:11:47,0:26:30
4,285,NimKik Skulls,132,1:26:53,0:15:06
5,285,NimKik Skulls,140,1:41:00,0:14:07
6,285,NimKik Skulls,141,2:15:35,0:34:35
7,285,NimKik Skulls,110,2:30:45,0:15:10
8,285,NimKik Skulls,109,2:45:03,0:14:18
9,285,NimKik Skulls,111,2:49:09,0:04:06


In [169]:
team_df = pd.read_csv('registration_list.csv', header=0)
score_df = pd.read_csv('Team_place_time_score.csv', header=None)
other_df = pd.read_csv('Number_of_Visits_to_each_Control_by_Course_.csv', header=0)

In [170]:
team_df['test'] = team_df['Team Name'].map(lambda x: ''.join(x.split(' ')).strip())

In [171]:
team_df.head(10)

,Team Name,Category,Size,Borrow,time_limit,test
0,BeeRingers,"Masters, Mixed",2,Y,6,BeeRingers
1,MMM,"Masters, Men",2,Y,6,MMM
2,High and Dry,"Masters, Men",2,Y,6,HighandDry
3,Team Spang,"Masters, Mixed",2,N,6,TeamSpang
4,Yaki Barak,"Masters, Men",1,N,6,YakiBarak
5,Beavis & Butthead,"Masters, Men",2,Y,6,Beavis&Butthead
6,Gordon Huang,"Elite, Men",1,Y,6,GordonHuang
7,Mountain Meisters,"Elite, Men",2,Y,6,MountainMeisters
8,Nightcrawlers,"Veterans, Mixed",2,N,6,Nightcrawlers
9,Victorious Seekers,"Elite, Mixed",2,Y,6,VictoriousSeekers


In [172]:
score_df.columns = ['Rank', 'Name', 'Class', 'Club', 'Time', 'Score', 'Gross Score', 'Penalty', 'time_limit']

In [173]:
score_df['team ID'] = score_df['Name'].map(lambda x: int(x[:3]))
score_df['Team Name'] = score_df['Name'].map(lambda x: x[4:].strip())
score_df['test'] = score_df['Team Name'].map(lambda x: ''.join(x.split(' ')).strip())

In [174]:
score_df[score_df['Team Name'].str.contains('Cycle')]

,Rank,Name,Class,Club,Time,Score,Gross Score,Penalty,time_limit,team ID,Team Name,test
17,17,459 American Adventure Sports Cycle Shop,EB6,NaN,NaN,710,710,0,6,459,American Adventure Sports Cycle Shop,AmericanAdventureSportsCycleShop


In [175]:
test_df = pd.merge(team_df, score_df, on='test', how='right')

In [176]:
test_df[test_df['Size'].isnull()]

,Team Name_x,Category,Size,Borrow,time_limit_x,test,Rank,Name,Class,Club,Time,Score,Gross Score,Penalty,time_limit_y,team ID,Team Name_y
69,NaN,NaN,NaN,NaN,NaN,FindersKeepers,14,406 Finders Keepers,EM6,NaN,NaN,740,740,0,6,406,Finders Keepers
70,NaN,NaN,NaN,NaN,NaN,AmericanAdventureSportsCycleShop,17,459 American Adventure Sports Cycle Shop,EB6,NaN,NaN,710,710,0,6,459,American Adventure Sports Cycle Shop
71,NaN,NaN,NaN,NaN,NaN,Team3DZ,7,286 Team 3DZ,EM3,NaN,NaN,180,180,0,3,286,Team 3DZ
72,NaN,NaN,NaN,NaN,NaN,TeamImmelmann,10,276 Team Immelmann,EB3,NaN,NaN,120,120,0,3,276,Team Immelmann


In [177]:
test_df.loc[test_df['team ID'] == 459, 'Borrow'] = 'Y'
test_df.loc[test_df['team ID'] == 459, 'Size'] = 4
test_df.loc[test_df['team ID'] == 286, 'Borrow'] = 'Y'
test_df.loc[test_df['team ID'] == 286, 'Size'] = 5
test_df.loc[test_df['team ID'] == 276, 'Borrow'] = 'Y'
test_df.loc[test_df['team ID'] == 276, 'Size'] = 2

In [178]:
test_df = test_df[test_df['Size'].notnull()]

In [179]:
test_df = test_df.drop('Team Name_x', 1)
test_df = test_df.drop('time_limit_x', 1)
test_df = test_df.drop('Name', 1)
test_df = test_df.rename(columns={'Team Name_y': 'Team Name', 'time_limit_y': 'Time Limit'})

In [181]:
test_df.head()

,Category,Size,Borrow,test,Rank,Class,Club,Time,Score,Gross Score,Penalty,Time Limit,team ID,Team Name
0,"Masters, Mixed",2,Y,BeeRingers,10,MB6,NaN,NaN,780,780,0,6,401,BeeRingers
1,"Masters, Men",2,Y,HighandDry,NaN,MM6,NaN,NaN,400,400,0,6,403,High and Dry
2,"Masters, Men",2,Y,HighandDry,53,MM6,NaN,NaN,-180,330,510,6,403,High and Dry
3,"Masters, Mixed",2,N,TeamSpang,NaN,MB6,WPOC,NaN,790,790,0,6,404,Team Spang
4,"Masters, Men",1,N,YakiBarak,37,MM6,WPOC,NaN,500,500,0,6,405,Yaki Barak


In [182]:
cols = test_df.columns.tolist()
cols = cols[-2:] + cols[:-2]
temp = cols[-4]
del cols[-4]
cols.append(temp)
temp = cols[-2]
del cols[-2]
cols.insert(2, temp)
test_df = test_df[cols]

In [183]:
test_df.head()

,team ID,Team Name,Time Limit,Category,Size,Borrow,test,Rank,Class,Club,Time,Gross Score,Penalty,Score
0,401,BeeRingers,6,"Masters, Mixed",2,Y,BeeRingers,10,MB6,NaN,NaN,780,0,780
1,403,High and Dry,6,"Masters, Men",2,Y,HighandDry,NaN,MM6,NaN,NaN,400,0,400
2,403,High and Dry,6,"Masters, Men",2,Y,HighandDry,53,MM6,NaN,NaN,330,510,-180
3,404,Team Spang,6,"Masters, Mixed",2,N,TeamSpang,NaN,MB6,WPOC,NaN,790,0,790
4,405,Yaki Barak,6,"Masters, Men",1,N,YakiBarak,37,MM6,WPOC,NaN,500,0,500
